In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 8, 5
plt.rcParams['font.size'] = 12
plt.rcParams['savefig.format'] = 'pdf'
sns.set_style('darkgrid')

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import stats

Определим фильтр и функцию для его применения.

In [11]:
def apply_filter(image, filter):
    res = np.array([
        [0, 0],
        [0, 0]
    ], dtype='float32')
    for i in [0, 1]:
        for j in [0, 1]:
            res[i, j] = np.sum(image[i:i + 2, j:j + 2] * filter)
    return res

filter = np.array([
    [1, -1],
    [2, 1],
], dtype='float32')

Нагенерируем кучу картинок, применим свёртку, получим вектора.

In [86]:
reses = []
for i in tqdm(range(200000)):
    img = np.zeros(shape=[3, 3], dtype='float32')
    for i in range(3):
        for j in range(3):
            img[i, j] = stats.norm.rvs(loc=0, scale=1)
    res = apply_filter(img, filter).reshape([-1])
    reses.append(res)
reses = np.array(reses)

100%|██████████| 200000/200000 [00:23<00:00, 8612.96it/s]


In [88]:
reses.shape

(200000, 4)

Посчитаем матрицу ковариации и найдём сумму её элементов. Без понятия, что это и как корректно считать, поэтому два способа.

In [89]:
np.cov(reses, rowvar=False, ddof=1).sum()

34.19643897733931

In [90]:
z = np.vstack((reses, reses))
c = np.cov(z.T)
c

array([[ 7.02421988,  1.03199283,  0.99369191,  1.02400272],
       [ 1.03199283,  7.02904535, -2.00978998,  1.02881578],
       [ 0.99369191, -2.00978998,  7.01792209,  0.98931481],
       [ 1.02400272,  1.02881578,  0.98931481,  7.00911005]])

In [91]:
c.sum()

34.19635348602816

Получаем что-то около 34, все значения матрицы ковариации почти целочисленные, что соответствует условию. 